In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Load the dataset and separate it into training and testing sets
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
mnist_dataset = mnist_bldr.as_dataset(shuffle_files=False)

mnist_train = mnist_dataset['train']
mnist_test = mnist_dataset['test']

In [3]:
# Preprocess the training and testing sets and create a validation set
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20

# Normalize the images in both sets
mnist_train = mnist_train.map(lambda item: (tf.cast(item['image'], tf.float32) / 255.0, tf.cast(item['label'], tf.int32)))
mnist_test = mnist_test.map(lambda item: (tf.cast(item['image'], tf.float32) / 255.0, tf.cast(item['label'], tf.int32)))

tf.random.set_seed(1)
mnist_train = mnist_train.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=False)

# Separate the training set into a training and validation set
mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE)
mnist_train = mnist_train.skip(10000).batch(BATCH_SIZE)

In [4]:
# Create the CNN
model = tf.keras.Sequential()

# Add 1st convolution layer and pooling layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), padding="same", data_format="channels_last", name="conv_1", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_1"))

# Add 2nd convolution layer and pooling layer
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), padding="same", name="conv_2", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_2"))

# Flatten the current output layer
model.add(tf.keras.layers.Flatten())

# Add 2 dense layers with a dropout layer in between
model.add(tf.keras.layers.Dense(units=1024, name='fc_1', activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(units=10, name='fc_2', activation='softmax'))

In [5]:
# Compile the model and view its summary
tf.random.set_seed(1)

model.build(input_shape=(None, 28, 28, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                      │ (None, 28, 28, 32)          │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool_1 (MaxPooling2D)                │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_2 (Conv2D)                      │ (None, 14, 14, 64)          │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool_2 (MaxPooling2D)                │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc_1 (Dense)                         │ (None, 1024)                │       3,212,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc_2 (Dense)                         │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,274,634 (12.49 MB)

 Trainable params: 3,274,634 (12.49 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
history = model.fit(mnist_train, epochs=NUM_EPOCHS, validation_data=mnist_valid, shuffle=True)

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 43ms/step - accuracy: 0.8952 - loss: 0.3229 - val_accuracy: 0.9776 - val_loss: 0.0728
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.9846 - loss: 0.0489 - val_accuracy: 0.9879 - val_loss: 0.0391
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 35s 44ms/step - accuracy: 0.9900 - loss: 0.0335 - val_accuracy: 0.9904 - val_loss: 0.0357
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.9919 - loss: 0.0262 - val_accuracy: 0.9890 - val_loss: 0.0367
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.9930 - loss: 0.0209 - val_accuracy: 0.9898 - val_loss: 0.0387
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - accuracy: 0.9951 - loss: 0.0151 - val_accuracy: 0.9885 - val_loss: 0.0409
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.9962 - loss: 0.0117 - val_accuracy: 0.9896 - val_loss: 0.0414
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 43ms/step - accuracy: 0.9964 - loss: 0.0108 - 

In [7]:
# Plot the graphs of the CNN's loss and accuracy scores
hist = history.history
x_arr = np.arange(len(hist['loss'])) + 1

fig = plt.figure(figsize=(12, 4))

# Plot the the 'Loss' graph
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist['loss'], '-o', label="Train Loss")
ax.plot(x_arr, hist['val_loss'], '--<', label="Validation Loss")
ax.legend(fontsize=15)

# Plot the 'Accuracy' graph
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist['accuracy'], '-o', label="Train Accuracy")
ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation Accuracy')
ax.legend(fontsize=15)
plt.show()

In [8]:
# Evaluate the results of the training
test_results = model.evaluate(mnist_test.batch(20))
print('Test Accuracy: {:.2f}%'.format(test_results[1] * 100))

500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9913 - loss: 0.0414
Test Accuracy: 99.15%


In [9]:
# Collect predictions on a sample batch from the testing set
batch_test = next(iter(mnist_test.batch(12)))
preds = model(batch_test[0])

# Reshape 'preds' to be a list of integers
preds = tf.argmax(preds, axis=1)

In [10]:
# Display the sample set with the model's predictions
fig = plt.figure(figsize=(12, 4))

for i in range(12):
    ax = fig.add_subplot(2, 6, i + 1)
    ax.set_xticks([]); ax.set_yticks([])
    img = batch_test[0][i, :, :, 0]
    ax.imshow(img, cmap='gray_r')
    ax.text(0.9, 0.1, '{}'.format(preds[i]), size=15, color='blue', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

plt.show()